<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Какие-артисты-чаще-других-попадали-в-годовой-топ-100-песен-по-популярности?" data-toc-modified-id="Какие-артисты-чаще-других-попадали-в-годовой-топ-100-песен-по-популярности?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Какие артисты чаще других попадали в годовой топ-100 песен по популярности?</a></span><ul class="toc-item"><li><span><a href="#НЕ-ОТНОСИТСЯ-К-ЗАДАНИЮ" data-toc-modified-id="НЕ-ОТНОСИТСЯ-К-ЗАДАНИЮ-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>НЕ ОТНОСИТСЯ К ЗАДАНИЮ</a></span></li></ul></li><li><span><a href="#Для-каждой-аудиохарактеристики-вывести-топ-3-артистов-по-среднему-значению-каждой-аудиохарактеристики-его-песен.-Дополнительно:-отнормировать-на-среднее-значение-аудиохарактеристики-в-год-выхода-песен." data-toc-modified-id="Для-каждой-аудиохарактеристики-вывести-топ-3-артистов-по-среднему-значению-каждой-аудиохарактеристики-его-песен.-Дополнительно:-отнормировать-на-среднее-значение-аудиохарактеристики-в-год-выхода-песен.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Для каждой аудиохарактеристики вывести топ 3 артистов по среднему значению каждой аудиохарактеристики его песен. Дополнительно: отнормировать на среднее значение аудиохарактеристики в год выхода песен.</a></span></li><li><span><a href="#Какие-артисты-дольше-других-подряд-держались-в-ежегодном-топ-100-песен-по-популярности?" data-toc-modified-id="Какие-артисты-дольше-других-подряд-держались-в-ежегодном-топ-100-песен-по-популярности?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Какие артисты дольше других подряд держались в ежегодном топ-100 песен по популярности?</a></span></li></ul></div>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.appName('hm3').getOrCreate()

In [3]:
df_pyspark = spark.read.csv('C:\\Users\\sysn1\\OneDrive\\Рабочий стол\\SQL ssis\\data.csv', header = True, inferSchema= True)

In [4]:
tracks = spark.read.option("header", "true") \
        .option("escape", '"') \
        .option("InferSchema", "true") \
        .csv("C:\\Users\\sysn1\\OneDrive\\Рабочий стол\\SQL ssis\\data.csv") \
        .withColumn("release_year", substring("release_date", 1, 4).cast(IntegerType())) \
        .withColumn("array_artist", split(regexp_replace(col("artists"), "[\]\[\']", ""),", ")) \
        .cache()

tracks_exp=tracks.select(  
                            "name", 
                            "popularity",
                            "danceability",
                            "energy",
                            "speechiness",
                            "acousticness",
                            "liveness",
                            "valence",
                            "release_year",
                            "artists",
                            explode( col("array_artist") ).alias("name_artist")
                        ) #создаем отдельную таблицу с развернутым массивом артистов
                        
tracks_exp = tracks_exp.na.drop(subset = 'release_year')

In [78]:
tracks_exp.columns

['name',
 'popularity',
 'danceability',
 'energy',
 'speechiness',
 'acousticness',
 'liveness',
 'valence',
 'release_year',
 'artists',
 'name_artist']

### Какие артисты чаще других попадали в годовой топ-100 песен по популярности?

In [229]:
# Узнаем популярность каждого артиста за один год (начиная с 1922 года по 2016 год)
name_artist_popularity = tracks_exp.select('name_artist','popularity','release_year')\
    .groupBy('name_artist','release_year')\
    .agg(sum('popularity').alias('sum_popularity'))\
    .orderBy(col('release_year').asc(),col('sum_popularity').desc())

# Объявляем оконную функцию. Задаем row_number, сортируем по уменьшению, чтобы дать самому популярному первое место.
# Нам нужны только первые 100 популярных за год.
windowSpec  = Window.partitionBy("release_year").orderBy(col('sum_popularity').desc())
top = name_artist_popularity.withColumn("top", row_number().over(windowSpec)).where('top  <= 100')

# Считаем сколько раз артист попал в топ 100 за весь период времени. 
top.select('name_artist').groupBy('name_artist').count().orderBy(col('count').desc()).show()


+--------------------+-----+
|         name_artist|count|
+--------------------+-----+
|Johann Sebastian ...|   30|
|Ludwig van Beethoven|   25|
|       Frank Sinatra|   25|
|New York Philharm...|   23|
|      Duke Ellington|   22|
|Wolfgang Amadeus ...|   21|
|     Frédéric Chopin|   19|
|      Jascha Heifetz|   18|
|      Eugene Ormandy|   17|
|         Glenn Gould|   16|
|                 Jr.|   15|
|    Charles Aznavour|   15|
|         Dean Martin|   15|
|        Van Morrison|   15|
|   Arthur Rubinstein|   14|
|Philadelphia Orch...|   14|
|       Grateful Dead|   14|
|   Vicente Fernández|   14|
|London Symphony O...|   14|
|         Miles Davis|   13|
+--------------------+-----+
only showing top 20 rows



#### НЕ ОТНОСИТСЯ К ЗАДАНИЮ

In [233]:
# Узнаем максимальную популярность за каждый год
release_year_max = tracks_exp.select('name_artist','popularity','release_year')\
    .groupBy('name_artist','release_year')\
    .sum('popularity')\
    .orderBy(col('release_year').asc(),col('sum(popularity)').desc())\
    .groupBy('release_year').agg(max('sum(popularity)').alias('max_popularity'))

# Узнаем популярность каждого артиста за один год (начиная с 1922 года)
name_artist_count = tracks_exp.select('name_artist','popularity','release_year')\
    .groupBy('name_artist','release_year')\
    .agg(sum('popularity').alias('sum_popularity'))\
    .orderBy(col('release_year').asc(),col('sum_popularity').desc())

In [234]:
# Джойним таблицы. Узнаем имя популярного артиста за каждый год.  
max_popularity = name_artist_count\
    .join(release_year_max, (name_artist_count.release_year == release_year_max.release_year) & (name_artist_count.sum_popularity == release_year_max.max_popularity))\
    .select('name_artist', release_year_max.release_year, 'max_popularity')\
    .orderBy(col('release_year').asc())

tracks_join.show(1000)

+--------------------+------------+--------------+
|         name_artist|release_year|max_popularity|
+--------------------+------------+--------------+
| Sergei Rachmaninoff|        1921|            22|
|         Louis Lynel|        1922|             0|
|         Jim Chapman|        1922|             0|
|      Victor Boucher|        1922|             0|
|           Perchicot|        1922|             0|
|             Fortugé|        1922|             0|
|            Georgius|        1922|             0|
| Jeanne Saint Bonnet|        1922|             0|
|      Adolphe Bérard|        1922|             0|
|       Francis Marty|        1922|             0|
|Luis Carlos Molin...|        1922|             0|
|         Félix Mayol|        1922|             0|
|   Maurice Chevalier|        1922|             0|
|               Dréan|        1922|             0|
|        Lucien Boyer|        1922|             0|
|         Greg Fieler|        1922|             0|
|         Mistinguett|        1

### Для каждой аудиохарактеристики вывести топ 3 артистов по среднему значению каждой аудиохарактеристики его песен. Дополнительно: отнормировать на среднее значение аудиохарактеристики в год выхода песен.

In [347]:
def df_rnd(x):
    return round(avg(x),2)

audio = tracks_exp.select("danceability","energy","speechiness","acousticness","liveness","valence","name_artist")\
    .where('release_year > 2010')\
    .groupBy("name_artist") \
    .agg(df_rnd("danceability").alias("danceability_avg"), df_rnd("energy").alias("energy_avg"), \
         df_rnd("speechiness").alias("speechiness_avg"), df_rnd("acousticness").alias("acousticness_avg"),\
         df_rnd("liveness").alias("liveness_avg"), df_rnd("valence").alias("valence_avg"))
audio.show()


+--------------------+----------------+----------+---------------+----------------+------------+-----------+
|         name_artist|danceability_avg|energy_avg|speechiness_avg|acousticness_avg|liveness_avg|valence_avg|
+--------------------+----------------+----------+---------------+----------------+------------+-----------+
|        Eden Project|            0.51|      0.75|            0.1|            0.12|        0.87|       0.28|
|       Petit Biscuit|            0.68|       0.4|           0.06|            0.84|         0.1|       0.36|
|         Los Rehenes|            0.71|      0.55|           0.06|            0.31|        0.21|       0.95|
|     Giorgio Moroder|            0.53|       0.6|           0.03|            0.29|        0.42|        0.5|
|        Britt Nicole|            0.54|      0.75|           0.06|            0.18|        0.18|       0.65|
| Secondhand Serenade|            0.43|      0.31|           0.03|            0.65|        0.06|       0.38|
|    Tijuana Panthe

In [348]:
#Запускаем оконную функцию
def windowSpec(x):
    return Window.partitionBy().orderBy(col(x).desc())

#Сортируем по убыванию и даем нумерацию каждой аудиохарактеристике, чтобы отобрать ТОП-3.
danceability_avg = audio.select(col('name_artist').alias('danceability'), 'danceability_avg')\
    .withColumn('top',row_number().over(windowSpec('danceability_avg')))\
    .where('top < 4')

energy_avg = audio.select(col('name_artist').alias('energy'), 'energy_avg')\
    .withColumn('top',row_number().over(windowSpec('energy_avg')))\
    .where('top < 4')

speechiness_avg = audio.select(col('name_artist').alias('speechiness'), 'speechiness_avg')\
    .withColumn('top',row_number().over(windowSpec('speechiness_avg')))\
    .where('top < 4')

acousticness_avg = audio.select(col('name_artist').alias('acousticness'), 'acousticness_avg')\
    .withColumn('top',row_number().over(windowSpec('acousticness_avg')))\
    .where('top < 4')

liveness_avg = audio.select(col('name_artist').alias('liveness'), 'liveness_avg')\
    .withColumn('top',row_number().over(windowSpec('liveness_avg')))\
    .where('top < 4')

valence_avg = audio.select(col('name_artist').alias('valence'), 'valence_avg')\
    .withColumn('top',row_number().over(windowSpec('valence_avg')))\
    .where('top < 4')

In [376]:
#Джойним полученные ТОП-3 и выводим результат.
audio_all_join = danceability_avg\
    .join(energy_avg, 'top','inner')\
    .join(speechiness_avg, 'top', 'inner')\
    .join(acousticness_avg,'top', 'inner')\
    .join(liveness_avg,'top', 'inner')\
    .join(valence_avg,'top', 'inner')\
    .select('top', 'danceability', 'energy', 'speechiness', 'acousticness', 'liveness','valence').cache()

audio_all_join.show()

+---+--------------------+--------------+-----------+------------+---------------+--------------+
|top|        danceability|        energy|speechiness|acousticness|       liveness|       valence|
+---+--------------------+--------------+-----------+------------+---------------+--------------+
|  1|Los Armadillos de...|     St. Lucia|  A$AP Ferg|  Erik Satie|ryo (supercell)|Banda El Limon|
|  2|            Don Aero|         Trapt|    Gunplay|Alena Cherny|   Darlene Love|   Los Rehenes|
|  3|              Zimple|Linked Horizon|     Apache| Max Richter|   Eden Project|       Joywave|
+---+--------------------+--------------+-----------+------------+---------------+--------------+



### Какие артисты дольше других подряд держались в ежегодном топ-100 песен по популярности?

In [89]:
# Узнаем популярность каждого артиста за один год (начиная с 1922 года по 2016 год)
name_artist_popularity = tracks_exp.select('name_artist','popularity','release_year')\
    .groupBy('name_artist','release_year')\
    .agg(sum('popularity').alias('sum_popularity'))\
    .orderBy(col('release_year').asc(),col('sum_popularity').desc())

#Объявляем оконную функцию.
windowSpec  = Window.partitionBy("release_year").orderBy(col('sum_popularity').desc())
top = name_artist_popularity.withColumn("top", row_number().over(windowSpec)).where('top  <= 100').cache()

In [90]:
# https://professorweb.ru/my/sql-server/window-functions/level3/3_12.php - объяснение, как считать диапозон.
windowSpec = Window.partitionBy("name_artist").orderBy(col('release_year').asc())
windowSpec_dns = Window.partitionBy()
top_new = top.withColumn('drnk', dense_rank().over(windowSpec))\
    .withColumn('diff', expr('release_year - drnk')).cache()

In [93]:
top_new.groupBy('name_artist','diff')\
    .agg(min(col('release_year')).alias('start_year'),max(col('release_year')).alias('end_year'))\
    .withColumn('diff_year', expr('end_year -start_year '))\
    .where('diff_year > 0')\
    .orderBy(col('diff_year').desc(), col('start_year').asc())\
    .show(1000)

+--------------------+----+----------+--------+---------+
|         name_artist|diff|start_year|end_year|diff_year|
+--------------------+----+----------+--------+---------+
|       Frank Sinatra|1949|      1955|    1969|       14|
|Johann Sebastian ...|1942|      1954|    1964|       10|
|       Benny Goodman|1929|      1932|    1941|        9|
|         Glenn Gould|1952|      1955|    1964|        9|
|       Hank Williams|1976|      1979|    1988|        9|
|                 Jr.|1974|      1979|    1988|        9|
|   Maurice Chevalier|1920|      1921|    1929|        8|
|       John Coltrane|1956|      1957|    1965|        8|
|        Teddy Wilson|1930|      1933|    1940|        7|
|      Billie Holiday|1929|      1933|    1940|        7|
|Ludwig van Beethoven|1935|      1941|    1948|        7|
|            Georgius|1920|      1921|    1927|        6|
|         Dean Martin|1951|      1956|    1962|        6|
|       Sonny Rollins|1952|      1953|    1958|        5|
|        Dave 